In [1]:
import cryptocompare
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta
plt.rcParams["figure.figsize"] = (18,7)

In [23]:
# how much data you want
limit_data = 200 #(hours)
# What you wanna trade
coin = 'ETH'
# Base currency 
currency = 'BUSD'

In [24]:
data = cryptocompare.get_historical_price_hour(coin, curr=currency, limit= limit_data, exchange='Binance')

In [25]:
# mean = []
time = []
high = []
low = []
open1 = []
close1 = []
volume = []
for i in range(len(data)):
    hi = data[i]['high']
    lo = data[i]['low']
    op = data[i]['open']
    cl = data[i]['close']
    vl = data[i]['volumefrom']
#     mean.append((hi+lo)/2)
    time.append(i)
    high.append(hi)
    low.append(lo)
    open1.append(op)
    close1.append(cl)
    volume.append(vl)
#df = pd.DataFrame(list(zip(pd.Series(pd.date_range('2000', freq='H', periods=limit_data)),high,low,open1,close1,volume,mean)), columns =['Date','high','low','open','close','volumefrom', 'mean']) 

time_delta = timedelta(hours=limit_data, minutes=0, seconds=0, microseconds=0) # minus hours 
date_at_start = datetime.now() - time_delta
days = pd.date_range(date_at_start, date_at_start + timedelta(limit_data/24), freq='H') # if freq is in hours then divide limit_data by 24

df = pd.DataFrame({'Date': days,'Open':open1, 'High': high, 'Low' : low, 'Close': close1, 'Volume' : volume})
df = df.set_index('Date')

In [26]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2020-12-30 14:55:00.758987,736.15,742.54,732.77,741.86,13584.05
2020-12-30 15:55:00.758987,741.86,758.69,740.96,753.25,34410.74
2020-12-30 16:55:00.758987,753.25,755.56,747.38,751.82,16954.80
2020-12-30 17:55:00.758987,751.82,755.85,741.39,747.85,15090.40
2020-12-30 18:55:00.758987,747.85,751.16,742.87,744.93,9184.70
...,...,...,...,...,...
2021-01-07 18:55:00.758987,1255.75,1260.41,1238.31,1246.14,9535.17
2021-01-07 19:55:00.758987,1246.14,1246.69,1202.41,1225.91,27621.55
2021-01-07 20:55:00.758987,1225.91,1233.18,1183.62,1184.30,19127.90


In [27]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import itertools
from backtesting.test import SMA

t1 = range(10)
t2 = range(20,40,2)
itemlist = list(itertools.product(t1,t2))

max_index = -1
max_ret = 0
for item in itemlist:
    class Smacross(Strategy):
        def init(self):
            price_close = self.data.Close
            self.ma1 = self.I(SMA, price_close, item[0])    #7 and 39 gives 54% return SUSHI hourly chart
            self.ma2 = self.I(SMA, price_close, item[1])


        def next(self):
            self.data.Volume
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt = Backtest(df, Smacross, commission=.005, exclusive_orders=True)
    stats = bt.run()
    #print(stats["Return [%]"])
    if stats["Return [%]"]>max_ret:
        max_ret = stats["Return [%]"]
        max_index = item
bt.plot()

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: 'Can't superimpose OHLC data with rule 'None'(index datetime resolution: 'microsecond'). Skipping.


In [28]:
print(max_index,max_ret)

(8, 28) 67.50044600000008
